# 构建图的一些思路

用ip地址前三位，客户经理，地址

In [ ]:
import igraph
import pandas as pd
import networkx
import gc
from sklearn.feature_extraction.text import CountVectorizer
import igraph
import warnings
import networkx as nx
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import datetime
from ge import Node2Vec#可能需要额外安装包


#vocab = list(model.wv.vocab.keys())
def get_emb(model,sentences,size=128):
    
    emb=np.zeros(size)
    n=len(sentences)
    for word in sentences:
        try:
            emb+=model.wv.word_vec(word)
        except(KeyError):
            n-=1
    return emb/n


In [ ]:
df=data_op[['user','ip_3']].dropna().drop_duplicates().groupby('user')['ip_3'].apply(list).reset_index()
df.ip_3=df.ip_3.str.join(',')
cv=CountVectorizer()
results=cv.fit_transform(df.ip_3)
#这一步统计的是每个id下，各ip分别的出现次数
result=results*results.T#建立领接矩阵
result[np.diag_indices_from(result)]=0#主对角线归0
conn_indices = result.nonzero()#得到非0元素位置
weights = result[conn_indices]#非0的元素
weights=weights[0].tolist() ##注意一定要转list##############

##########然后就可以转pandas了
edges = zip(*conn_indices)
G = igraph.Graph(edges=edges, directed=False)

In [ ]:
tempg = G.get_edgelist()
g = networkx.Graph(G.get_edgelist()) 
del tempg;gc.collect()
fig = plt.figure(figsize=(15,15))
nx.draw(g)
plt.show()

In [ ]:
G.es['weight'] = weights[0]
G.vs['user']=df.user#设置主键
lpa=G.community_label_propagation()
df['lpa']=lpa.membership
df['pagerank']=G.pagerank(directed=False,weights='weight')
louvain=G.community_multilevel(weights='weight') ##louvain社区的发现
df['louvain']=louvain.membership
df['degree']=G.degree()
infomap=G.community_infomap(edge_weights='weight') #infomap社区的发现
df['infomap']=infomap.membership
df['closeness']=G.closeness()
df['eccentricity']=G.eccentricity()#离心率
df.to_csv('df_user_ip3.csv',index=False)

#需要注意的是对两个社区发现算法发现出来的社区可以继续进行聚合操作

In [ ]:
words=df.ip_3.str.split(',').values.tolist()
start = datetime.datetime.now()
#可以看做教科书级别的图嵌入和序列嵌入参数设置
model = Word2Vec(sentences = words, 
                 iter = 50, 
                 min_count = 1,# 必要设置
                 size = 128, 
                 workers = 4, 
                 sg = 1, 
                 hs = 0, 
                 negative = 5,#>0 则会采用negativesamping，用于设置多少个noise words
                 window = 9999999)#表示当前词与预测词在一个句子中的最大距离是多少

print("Time passed: " + str(datetime.datetime.now()-start))
model.save('ip_3_word2vec')

In [ ]:
def get_average(sentence):
    n=len(sentence)
    embedding=np.zeros(200)
    for word in sentence:
        
        try:
            embedding+=model.wv[word]
        except(KeyError):
            n-=1
    
    return embedding/n

In [ ]:
#把嵌入的向量作为特征拼接进原始数据
sentences=[]
for sentence in words:
    sentences.append(get_average(sentence))
sentences=pd.DataFrame(sentences)
sentences.columns=['ip_3_emb_'+str(item) for item in range(128)]
for col in sentences.columns:
    df[col]=sentences[col].values
df.to_csv('user_ip_3.csv',index=False)

In [ ]:
#把上面的代码进行集成
def graph(data,cols=['user','ip_3']):
    df=data[cols].dropna().drop_duplicates().groupby(cols[0])[cols[1]].apply(list).reset_index()#group成list
    df[cols[1]]=df[cols[1]].str.join(',')#用逗号分割并做成str格式
    cv=CountVectorizer(dtype=np.int8)
    results=cv.fit_transform(df[cols[1]])
    result=results.astype(np.int8).dot(results.astype(np.int8).T)
    result[np.diag_indices_from(result)]=0
    conn_indices = result.nonzero()
    weights = result[conn_indices]
    weights=weights[0].tolist() ##注意一定要转list##############

    edges = zip(*conn_indices)
    G = igraph.Graph(edges=edges, directed=False)
    G.es['weight'] = weights[0]
    G.vs['user']=df.user#这个地方是主键
    lpa=G.community_label_propagation()
    df[cols[1]+'_''lpa']=lpa.membership
    df[cols[1]+'_''pagerank']=G.pagerank(directed=False,weights='weight')
    louvain=G.community_multilevel(weights='weight') ##louvain社区的发现
    df[cols[1]+'_''louvain']=louvain.membership
    df[cols[1]+'_''degree']=G.degree()
    words=df[cols[1]].str.split(',').values.tolist()
    
    start = datetime.datetime.now()

    model = Word2Vec(sentences = words, 
                     iter = 50, 
                     min_count = 1,
                     size = 128, 
                     workers = 4, 
                     sg = 1, 
                     hs = 0, 
                     negative = 5, 
                     window = 9999999)

    print("Time passed: " + str(datetime.datetime.now()-start))
    model.save(cols[1]+'_word2vec')
    sentences=[]


    vocab = list(model.wv.vocab.keys())
    for sentence in words:
        sentences.append(get_emb(model,sentence))
    sentences=pd.DataFrame(sentences)
    sentences.columns=[cols[1]+'_emb_'+str(item) for item in range(128)]
    for col in sentences.columns:
        df[col]=sentences[col].values
    df.to_csv(cols[0]+'_'+cols[1]+'.csv',index=False)

In [ ]:
#这里用ge包的node2vec来快速实现
def just_emb(data,cols=['user','ip_3']):
    df=data[cols].dropna().drop_duplicates().groupby(cols[0])[cols[1]].apply(list).reset_index()
    df[cols[1]]=df[cols[1]].str.join(',')
    cv=CountVectorizer(dtype=np.int8)
    results=cv.fit_transform(df[cols[1]])
    result=[]
    for i in range(results.shape[0]):
        result.append(results[i]*results.T)
   # result=results.astype(np.int8).dot(results.astype(np.int8).T)
    result=np.concatenate(result)
    result[np.diag_indices_from(result)]=0
    conn_indices = result.nonzero()
    weights = result[conn_indices]
    weights=weights[0].tolist() ##注意一定要转list##############
    edges = zip(*conn_indices)
    
    g=pd.DataFrame(edges)
    g.columns=['source','target']
    g['weight']=weights[0]
    G = nx.from_pandas_edgelist(g,source= 'source',target= 'target', edge_attr='weight')

    
    start = datetime.datetime.now()
    model = Node2Vec(G, walk_length = 10, num_walks = 5,p = 0.25, q = 4, workers = 4,use_rejection_sampling=1)#init model
    model.sentences=pd.DataFrame(model.sentences).astype(str).values.tolist()
    model.train(window_size=5,iter=20)

    print("Time passed: " + str(datetime.datetime.now()-start))
#    model.save(cols[1]+'_word2vec')
    sentences=[]


    for sentence in df[cols[1]].str.split(',').values.tolist():
        sentences.append(get_emb(model.w2v_model,sentence))
    sentences=pd.DataFrame(sentences)
    sentences.columns=[cols[1]+'_net_emb_'+str(item) for item in range(128)]
    for col in sentences.columns:
        df[col]=sentences[col].values
    df.to_csv(cols[0]+'_'+cols[1]+'_net.csv',index=False)

In [ ]:
def just_emb(data,cols=['user','ip_3']):
    df=data[cols].dropna().drop_duplicates().groupby(cols[0])[cols[1]].apply(list).reset_index()
    df[cols[1]]=df[cols[1]].str.join(',')
    cv=CountVectorizer(dtype=np.int8)
    results=cv.fit_transform(df[cols[1]])
    result=[]
    for i in range(results.shape[0]):
        result.append(results[i]*results.T)
   # result=results.astype(np.int8).dot(results.astype(np.int8).T)
    result=np.concatenate(result)
    result[np.diag_indices_from(result)]=0
    conn_indices = result.nonzero()
    weights = result[conn_indices]
    weights=weights[0].tolist() ##注意一定要转list##############
    edges = zip(*conn_indices)
    
    g=pd.DataFrame(edges)
    g.columns=['source','target']
    g['weight']=weights[0]
    G = nx.from_pandas_edgelist(g,source= 'source',target= 'target', edge_attr='weight')

    
    start = datetime.datetime.now()
    model = Node2Vec(G, walk_length = 10, num_walks = 5,p = 0.25, q = 4, workers = 4,use_rejection_sampling=1)#init model
    model.sentences=pd.DataFrame(model.sentences).astype(str).values.tolist()
    model.train(window_size=5,iter=20)

    print("Time passed: " + str(datetime.datetime.now()-start))
#    model.save(cols[1]+'_word2vec')
    sentences=[]


    for sentence in df[cols[1]].str.split(',').values.tolist():
        sentences.append(get_emb(model.w2v_model,sentence))
    sentences=pd.DataFrame(sentences)
    sentences.columns=[cols[1]+'_net_emb_'+str(item) for item in range(128)]
    for col in sentences.columns:
        df[col]=sentences[col].values
    df.to_csv(cols[0]+'_'+cols[1]+'_net.csv',index=False

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

texts=["dog cat fish","dog cat cat","fish bird", 'bird'] # “dog cat fish” 为输入列表元素,即代表一个文章的字符串
cv = CountVectorizer()#创建词袋数据结构
cv_fit=cv.fit_transform(texts)
#上述代码等价于下面两行
#cv.fit(texts)
#cv_fit=cv.transform(texts)

print(cv.get_feature_names())    #['bird', 'cat', 'dog', 'fish'] 列表形式呈现文章生成的词典

print(cv.vocabulary_	)              # {‘dog’:2,'cat':1,'fish':3,'bird':0} 字典形式呈现，key：词，value:词频

['bird', 'cat', 'dog', 'fish']
{'dog': 2, 'cat': 1, 'fish': 3, 'bird': 0}


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
